# 1. Set up

In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

import re
import json
import gensim.downloader as api

!pip install langid
import langid

!pip install gensim
from gensim.models import Word2Vec

!pip install -U spacy
!python -m spacy download it_core_news_sm
import spacy

import random

  Using cached spacy-3.7.4-cp39-cp39-win_amd64.whl.metadata (27 kB)
  Using cached thinc-8.2.3-cp39-cp39-win_amd64.whl.metadata (15 kB)
  Using cached srsly-2.4.8-cp39-cp39-win_amd64.whl.metadata (20 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
Using cached spacy-3.7.4-cp39-cp39-win_amd64.whl (12.2 MB)
Using cached catalogue-2.0.10-py3-none-any.whl (17 kB)
Using cached srsly-2.4.8-cp39-cp39-win_amd64.whl (483 kB)
Using cached thinc-8.2.3-cp39-cp39-win_amd64.whl (1.5 MB)
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.2
    Uninstalling catalogue-1.0.2:
      Successfully uninstalled catalogue-1.0.2
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.7
    Uninstalling srsly-1.0.7:
      Successfully uninstalled srsly-1.0.7
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting uninstall: spacy
    Fou

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-use-md 0.4.3 requires spacy<3.1.0,>=3.0.3, but you have spacy 3.7.4 which is incompatible.
sense2vec 1.0.2 requires spacy<3.0.0,>=2.2.3, but you have spacy 3.7.4 which is incompatible.
Traceback (most recent call last):
  File "c:\Users\35193\miniconda3\envs\cuda_env\lib\runpy.py", line 188, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "c:\Users\35193\miniconda3\envs\cuda_env\lib\runpy.py", line 147, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "c:\Users\35193\miniconda3\envs\cuda_env\lib\runpy.py", line 111, in _get_module_details
    __import__(pkg_name)
  File "c:\Users\35193\miniconda3\envs\cuda_env\lib\site-packages\spacy\__init__.py", line 13, in <module>
    from . import pipeline  # noqa: F401
  File "c:\Users\35

TypeError: issubclass() arg 1 must be a class

# Find Distractores


# Fasttext

Trying wordembedings from Fast Text  https://fasttext.cc/docs/en/support.html#building-fasttext-python-module

In [3]:
# !git clone https://github.com/facebookresearch/fastText.git
# !cd fastText
# !sudo pip install .
# or :
# !sudo python setup.py install

In [4]:
# import fasttext.util
# fasttext.util.download_model('it', if_exists='ignore')  # English


In [ ]:
# !git clone https://github.com/facebookresearch/fastText.git
# !cd fastText
# !pip install fastText
import fasttext.util
#fasttext.util.download_model('it', if_exists='ignore')  # English
model = fasttext.load_model('cc.it.300.bin')

In [5]:
model.get_nearest_neighbors('numero ordinale')

[(0.69635009765625, 'dinale'),
 (0.6637539267539978, 'tecnicoeconomica'),
 (0.6575295329093933, 'numeroe'),
 (0.6210983991622925, 'ordinal'),
 (0.617618203163147, 'Ordinale'),
 (0.6075699925422668, 'Numeroordinale'),
 (0.6015861630439758, 'Numerocardinale'),
 (0.5819660425186157, 'estetologo'),
 (0.5808798670768738, 'Sindacatore'),
 (0.5799951553344727, 'anticardinale')]

In [6]:
model.get_analogies("numero ordinale", "carinale", "millesimo")

[(0.4903110861778259, 'decimillesimo'),
 (0.46558308601379395, 'millesimi'),
 (0.430698961019516, 'duecentesimo'),
 (0.4299042224884033, 'ordinalità'),
 (0.4258556365966797, 'centesimo'),
 (0.421135812997818, 'decimo'),
 (0.4166191816329956, 'trecentesimo'),
 (0.40764376521110535, 'decimillesimi'),
 (0.4020084738731384, 'cinquantesimo'),
 (0.40015530586242676, 'milionesimo')]

## Insert distractors into the dataset

In [7]:
with open("hypernym_discovery-task26-train-data.jsonl", 'r') as file:
        count = 0  # Counter to track processed pairs
        for line in file:
            if count >= 1:  # Break the loop if 5 pairs have been processed
                break
            pair = json.loads(line)
            id = pair["id"]
            choices = pair["choices"]
            label = pair["label"]
            golden_label = choices[label]
            print(f"\nTrue values: {choices[label]}")
            print(f"Predicted values: {model.get_nearest_neighbors(choices[label])}")
            # for i in range(3):
            distractor = random.choice(model.get_nearest_neighbors(choices[label]))
            print(f"Distractor {i + 1}: {distractor}")

            count += 1




True values: grado
Predicted values: [(0.6741050481796265, 'gradodi'), (0.6480050683021545, 'capace'), (0.641866147518158, 'capaci'), (0.5582722425460815, 'grado.'), (0.554225504398346, 'ingrado'), (0.5511019825935364, 'poter'), (0.5481618642807007, 'grado.Il'), (0.5384020209312439, 'grado-'), (0.5263723134994507, 'livello'), (0.5168741345405579, 'saper')]


NameError: name 'random' is not defined

In [ ]:
from scipy.spatial.distance import cosine
import numpy as np
import fasttext
import fasttext.util

# download an english model
# fasttext.util.download_model('en', if_exists='ignore')  # English
# model = fasttext.load_model('cc.it.300.bin')
print(distractor[1], golden_label)
# Getting word vectors for 'one' and 'two'.
embedding_1 = model.get_word_vector(distractor[1])
embedding_2 = model.get_word_vector(golden_label)

def cosine_similarity(embedding_1, embedding_2):
    # Calculate the cosine similarity of the two embeddings.
    sim = 1 - cosine(embedding_1, embedding_2)
    print('Cosine similarity: {:.2}'.format(sim))
    
# compare the embeddings
cosine_similarity(embedding_1, embedding_2)

saper grado
Cosine similarity: 0.41


In [18]:
import json
import random
from scipy.spatial.distance import cosine
import fasttext
import fasttext.util

# Load FastText model
# fasttext.util.download_model('en', if_exists='ignore')  # Download English model if needed
# model = fasttext.load_model('cc.it.300.bin')  # Load your model, specify the correct path

# Function that calculates cosine similarity
def cosine_similarity(embedding_1, embedding_2):
    return 1 - cosine(embedding_1, embedding_2)

count = 0  # Move count initialization outside of the loop

with open("hypernym_discovery-task26-train-data.jsonl", 'r') as file:
    for line in file: 
        if count < 3:  # Check count inside the loop
            pair = json.loads(line)
            choices = pair["choices"]
            label = pair["label"]
            golden_label = choices[label]
        
            print(f"\nGolden Label: {golden_label}")
            
            if isinstance(choices, list):
                choices = {str(idx): choice for idx, choice in enumerate(choices)}
            
            # Get word embedding for golden label
            golden_embedding = model.get_word_vector(golden_label)
            
            # Generate three distractors 
            distractors = set()
            while len(distractors) < 3:
                distractor = random.choice(list(choices.values()))
                if distractor != golden_label:
                    distractor_embedding = model.get_word_vector(distractor)
                    similarity = cosine_similarity(golden_embedding, distractor_embedding)
                    if 0.1 <= similarity <= 0.7: # Set the range of cosine similarity
                        distractors.add(distractor)
            
            # Print the distractors and calculate cosine similarities
            for i, distractor in enumerate(distractors):
                print(f"Distractor {i + 1}: {distractor}")
                
                # Get word embedding for distractor
                distractor_embedding = model.get_word_vector(distractor)
                
                # Calculate cosine similarity between golden label and distractor embeddings
                similarity = cosine_similarity(golden_embedding, distractor_embedding)
                print('Cosine similarity: {:.2}'.format(similarity))
            count += 1



Golden Label: grado
Distractor 1: formicolio
Cosine similarity: 0.16
Distractor 2: alternare
Cosine similarity: 0.3
Distractor 3: negativizzazione
Cosine similarity: 0.19

Golden Label: numero ordinale
Distractor 1: immagazzinare
Cosine similarity: 0.2
Distractor 2: solubile
Cosine similarity: 0.15
Distractor 3: autenticità
Cosine similarity: 0.13

Golden Label: frazione
Distractor 1: cristiano
Cosine similarity: 0.11
Distractor 2: elettromagnetico
Cosine similarity: 0.12
Distractor 3: serenza
Cosine similarity: 0.28
